In [1]:
println("start")

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-D4GN0MQF:4042
SparkContext available as 'sc' (version = 2.3.0, master = local[*], app id = local-1565719747321)
SparkSession available as 'spark'


start


# Imports

In [13]:
import org.apache.spark.sql.Row
import org.apache.spark.sql.types.StructType
import spark.implicits._

import org.apache.spark.sql.Row
import org.apache.spark.sql.types.StructType
import spark.implicits._


In [8]:
val a = Seq(
            ("10", "MILLER", "1300", "2017-11-03"),
            ("30", "Martin", "1250", "2017-11-21")
           ).toDF("dept_no","emp_name","sal","date")

a: org.apache.spark.sql.DataFrame = [dept_no: string, emp_name: string ... 2 more fields]


In [9]:
a.show

+-------+--------+----+----------+
|dept_no|emp_name| sal|      date|
+-------+--------+----+----------+
|     10|  MILLER|1300|2017-11-03|
|     30|  Martin|1250|2017-11-21|
+-------+--------+----+----------+



In [10]:
val lst = List("10", "Susan").toDF

lst: org.apache.spark.sql.DataFrame = [value: string]


In [11]:
val data = a.rdd.zip(lst.rdd).map { case (l, r) => Row.fromSeq(l.toSeq ++ r.toSeq) }
val schema = StructType(a.schema.fields ++ lst.schema.fields)
val solution = spark.createDataFrame(data, schema)

data: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[9] at map at <console>:30
schema: org.apache.spark.sql.types.StructType = StructType(StructField(dept_no,StringType,true), StructField(emp_name,StringType,true), StructField(sal,StringType,true), StructField(date,StringType,true), StructField(value,StringType,true))
solution: org.apache.spark.sql.DataFrame = [dept_no: string, emp_name: string ... 3 more fields]


In [12]:
solution.show

+-------+--------+----+----------+-----+
|dept_no|emp_name| sal|      date|value|
+-------+--------+----+----------+-----+
|     10|  MILLER|1300|2017-11-03|   10|
|     30|  Martin|1250|2017-11-21|Susan|
+-------+--------+----+----------+-----+



Apply function on column based on a different column value

In [15]:
var df = Seq(
  ("TAG1", "2019-06-21 01:16:00.0",621.0947),
  ("TAG1", "2019-06-21 01:16:00.0",621.0947),
  ("TAG1", "2019-06-21 01:16:00.0",621.0947),
  ("TAG1", "2019-06-21 01:16:00.0",619.9578),
  ("TAG2", "2019-06-21 01:29:00.0",767.5475),
  ("TAG2", "2019-06-21 01:29:00.0",768.9506),
  ("TAG2", "2019-06-21 01:29:00.0",770.8863), //Map(Tag1 -> avg, Tag2 -> max, Tag3 -> min)
  ("TAG3", "2019-06-21 01:16:00.0",203.7457)).toDF("TAG","Timestamp","Value")

df: org.apache.spark.sql.DataFrame = [TAG: string, Timestamp: string ... 1 more field]


In [38]:
val dif_aggregation = df.groupBy(
    "TAG", "Timestamp"
).agg(
    when(
        col("TAG") === "TAG1", avg("Value")
    ).otherwise(
        when(col("TAG") === "TAG2", max("Value")).otherwise(min("Value"))
    ).as("Value"))

dif_aggregation: org.apache.spark.sql.DataFrame = [TAG: string, Timestamp: string ... 1 more field]


In [39]:
dif_aggregation.show

+----+--------------------+----------+
| TAG|           Timestamp|     Value|
+----+--------------------+----------+
|TAG3|2019-06-21 01:16:...|  203.7457|
|TAG1|2019-06-21 01:16:...|620.810475|
|TAG2|2019-06-21 01:29:...|  770.8863|
+----+--------------------+----------+



res7: org.apache.spark.sql.DataFrame = [TAG: string, Timestamp: string ... 1 more field]
